# Mergin Maps Users Management

The Mergin Maps client allows you to manage your workspace users, their roles, and project-specific permissions. Every user has a workspace role and a project role. A project role is defined for a specific project and overrides the workspace role for that project. For more details, see the [permissions documentation](https://dev.merginmaps.com/docs/manage/permissions/).

See the [API reference](https://merginmaps.com/docs/dev/integration/) for more details about methods used in this notebook.

First let's install mergin maps client

In [ ]:
!pip install mergin-client

Login to Mergin Maps using your workspace user with `Owner` permission.

In [ ]:
# Use here your login username and password
LOGIN = "..."
PASSWORD = "..."

import mergin

client = mergin.MerginClient(login=LOGIN, password=PASSWORD)

Let's create a workspace and project as base for our users management example.

Set the `WORKSPACE` variable to your desired workspace name and the `PROJECT` variable to the name of the project created in your workspace.

In [ ]:
# Add here your existing workspace name and the new project name
WORKSPACE = "..."
PROJECT = "..."

# Create new workspace
# INFO: Only uncomment if you are able to create a new workspace. Mergin Maps free tier only allows for 1 workspace per user. In this case use your existing workspace.
# client.create_workspace(WORKSPACE)

# Create new project
client.create_project(project_name=PROJECT, namespace=WORKSPACE)

Create some users on your Mergin Maps example workspace from the provided example file in `01_users_assets/users.csv` with random permissions.

In [ ]:
# First, let's get workspace ID
WORKSPACE_ID = None
for p in client.workspaces_list():
  if p['name'] == WORKSPACE:
    WORKSPACE_ID = p['id']

print(f"WORKSPACE_ID: {WORKSPACE_ID}")

WORKSPACE_ID: 15639


We can now use the client to create multiple users at once in your workspace by importing them from a CSV file. In this case, we will use the example users with random choice of password and workspace role. Username is optional, Megin Maps will generate username from email address.

In [ ]:
from os.path import join
import csv
import string
import random

from mergin.common import WorkspaceRole

filename = "01_users_assets/users.csv"

with open(filename, mode="r", newline="", encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile)
    header = next(reader)  # Skip header
    for row in reader:
        username = row[0]
        email = row[1]
        # add new mergin maps user
        password = "".join(
            random.choices(string.ascii_uppercase + string.ascii_lowercase, k=10) + random.choices(string.digits, k=3)
        )
        client.create_user(
            email=email,
            password=password,
            workspace_id=WORKSPACE_ID,
            workspace_role=WorkspaceRole.READER,
            username=username,
        )
        print(f"User with email {email} created, password: {password}")

User with email jdoe@example.com created, password: PpWVMDIpGB164
User with email asmith@example.com created, password: yvhWPxvZkU230
User with email bwilliams@example.com created, password: ofgPmxerDW473
User with email cjohnson@example.com created, password: ktAhUKuOnu295
User with email omartin@example.com created, password: hIWhvYoNNh661


We can now list all users in workspace using `list_workspace_members` method.

In [15]:
members = client.list_workspace_members(WORKSPACE_ID)
print(members)

[{'email': 'asmith@example.com', 'id': 22597, 'username': 'asmith', 'workspace_role': 'reader'}, {'email': 'bwilliams@example.com', 'id': 22598, 'username': 'bwilliams', 'workspace_role': 'reader'}, {'email': 'cjohnson@example.com', 'id': 22599, 'username': 'cjohnson', 'workspace_role': 'reader'}, {'email': 'jdoe@example.com', 'id': 22596, 'username': 'jdoe', 'workspace_role': 'reader'}, {'email': 'marcel.kocisek+r22overviews@lutraconsulting.co.uk', 'id': 12800, 'username': 'r22overviews', 'workspace_role': 'owner'}, {'email': 'omartin@example.com', 'id': 22600, 'username': 'omartin', 'workspace_role': 'reader'}]


Let's change workspace permission level for a specific user to `EDITOR`.

In [ ]:
new_editor = members[0]
print(f"Changing role of user {new_editor['username']} to editor")
client.update_workspace_member(
    user_id=new_editor.get("id", 0), workspace_id=WORKSPACE_ID, workspace_role=WorkspaceRole.EDITOR
)

Changing role of user asmith to editor


{'email': 'asmith@example.com',
 'id': 22597,
 'projects_roles': [],
 'username': 'asmith',
 'workspace_role': 'editor'}

The user is now an editor for the entire workspace and every project within it. If you want to change the user's role to 'writer' for a specific project, you need to add them as a project collaborator.

In [ ]:
from mergin.common import ProjectRole

# find project id of project used in this example
PROJECT_ID = None
for p in client.projects_list(namespace=WORKSPACE):
    if p["name"] == PROJECT:
        PROJECT_ID = p["id"]


client.add_project_collaborator(
    user=new_editor.get("email", ""), project_id=PROJECT_ID, project_role=ProjectRole.WRITER
)

{'email': 'asmith@example.com',
 'id': 22597,
 'project_role': 'writer',
 'role': 'writer',
 'username': 'asmith',
 'workspace_role': 'editor'}

We can now see that the user has beed added to project with different role than the `workspace_role`. Project role is now `writer`.

We can now upgrade user to full control over project.

In [22]:
client.update_project_collaborator(
    user_id=new_editor.get("id", 0), project_id=PROJECT_ID, project_role=ProjectRole.OWNER
)

{'email': 'asmith@example.com',
 'id': 22597,
 'project_role': 'owner',
 'role': 'owner',
 'username': 'asmith',
 'workspace_role': 'editor'}

To simply remove that user from project and use his default role, you can use the `remove_project_collaborator` method.

In [23]:
client.remove_project_collaborator(
    user_id=new_editor.get("id", 0), project_id=PROJECT_ID
)

Check which users have access to the project using `list_project_collaborators` method.

In [24]:
client.list_project_collaborators(project_id=PROJECT_ID)

[{'email': 'asmith@example.com',
  'id': 22597,
  'project_role': None,
  'role': 'editor',
  'username': 'asmith',
  'workspace_role': 'editor'},
 {'email': 'bwilliams@example.com',
  'id': 22598,
  'project_role': None,
  'role': 'reader',
  'username': 'bwilliams',
  'workspace_role': 'reader'},
 {'email': 'cjohnson@example.com',
  'id': 22599,
  'project_role': None,
  'role': 'reader',
  'username': 'cjohnson',
  'workspace_role': 'reader'},
 {'email': 'jdoe@example.com',
  'id': 22596,
  'project_role': None,
  'role': 'reader',
  'username': 'jdoe',
  'workspace_role': 'reader'},
 {'email': 'marcel.kocisek+r22overviews@lutraconsulting.co.uk',
  'id': 12800,
  'project_role': None,
  'role': 'owner',
  'username': 'r22overviews',
  'workspace_role': 'owner'},
 {'email': 'omartin@example.com',
  'id': 22600,
  'project_role': None,
  'role': 'reader',
  'username': 'omartin',
  'workspace_role': 'reader'}]

Remove a user from an workspace to completely remove them from every project.

In [25]:
client.remove_workspace_member(user_id=new_editor.get("id", 0), workspace_id=WORKSPACE_ID)